# XGBoost分類器(XGBoost Classifier)
Model Definition: マルチクラス分類に最適化されたXGBoost分類器を使用。`eval_metric='mlogloss'`を指定し、高精度な予測を実現。

Training: サンプリングされたトレーニングデータセットで学習を実施。

Prediction: 検証セットで予測を行い、モデルの性能を評価。

Evaluation: XGBoostモデルの性能を分析するため、分類レポートと精度指標を表示。また、特徴量の重要度も分析可能。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import os

# Define dataset filenames
train_file = "train_1_split.csv"
val_file = "val_1_split.csv"

# Construct dynamic paths based on the current working directory
current_dir = os.getcwd()
train_path = os.path.join(current_dir, "/content/drive/MyDrive/image_analytics/data/features", train_file)
val_path = os.path.join(current_dir, "/content/drive/MyDrive/image_analytics/data/features", val_file)

# Optional check if paths exist
if not os.path.isfile(train_path):
    print(f"Warning: {train_path} が見つかりません")
if not os.path.isfile(val_path):
    print(f"Warning: {val_path} が見つかりません")

# Load the training and validation datasets
train = pd.read_csv(train_path)
val = pd.read_csv(val_path)

# Separate features and labels for the training and validation sets
X_train = train.iloc[:, 2:].astype(np.float32)
y_train = train['label']
X_val = val.iloc[:, 2:].astype(np.float32)
y_val = val['label']

# Sample a subset for training
X_train_sample, _, y_train_sample, _ = train_test_split(X_train, y_train, train_size=50000, random_state=42, stratify=y_train)

# Confirm dimensions
print("Training Features Shape:", X_train.shape)
print("Training Labels Shape:", y_train.shape)
print("Validation Features Shape:", X_val.shape)
print("Validation Labels Shape:", y_val.shape)


## XGBoost Model: Training, Prediction, and Evaluation
ここでは、XGBoost分類器を実装し、検証データでその性能を評価します。
1. **XGBoost分類器の定義**:
  - 再現性を確保するために`random_state`を設定。
  - 警告を避け、マルチクラス分類を最適化するために`use_label_encoder`を無効化し、`eval_metric='mlogloss'`を指定。

2. **モデルのトレーニング**: 分類器はサンプリングされたトレーニングデータセットを使用して学習されます。

3. **予測の実行**: 検証データセットに対して予測を生成しました。

4. **分類器の評価**: 精度と各クラスの適合率、再現率、F1スコアを含む包括的な分類レポートを出力。

In [ ]:
# Define the XGBoost Classifier with default parameters
xgb_clf = XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss')

# Train the classifier on the sampled training data
xgb_clf.fit(X_train_sample, y_train_sample)

# Make predictions on the validation data
xgb_y_pred = xgb_clf.predict(X_val)

# Define the new path for saving the model inside the models folder
model_save_path = "/content/drive/MyDrive/image_analytics/models/svm_classifier.pkl"

# Save the trained model
joblib.dump(xgb_clf, model_save_path)

print(f"Model saved to {model_save_path}.")

# Evaluate the model
print("XGBoost Classifier Accuracy:", accuracy_score(y_val, xgb_y_pred))
print("\nClassification Report:\n", classification_report(y_val, xgb_y_pred))


### Model Evaluation Metrics

モデルを評価するための包括的な指標のセットを計算し、表示します。

1. **精度（**ACCURACY**）**: 正しい予測の全体的な割合。
2. **適合率（**PRECISION**）（加重）**: クラスの不均衡を考慮した、全クラスの適合率の加重平均。
3. **再現率（**RECALL**）（加重）**: 全クラスの再現率の加重平均。
4. **F1スコア（**F1 SCORE**）（加重）**: 適合率と再現率のバランスを取る、全クラスのF1スコアの加重平均。


In [ ]:
# Display model metrics and classification report for XGBoost

from sklearn.metrics import classification_report, accuracy_score

# Make Calculate and display accuracy
accuracy_xgb = accuracy_score(y_val, xgb_y_pred)
print(f"XGBoost Classifier Accuracy: {accuracy_xgb:.4f}")

# Generate and display classification report
xgb_classification_report = classification_report(y_val, xgb_y_pred)
print("\nClassification Report:\n", xgb_classification_report)


### XGBoost Model Performance Visualization
1. **混同行列作成（**CONFUSION MATRIX**）**:
  - 各クラスの真の予測と誤った予測の分布を示す混同行列のヒートマップ。
  - 分類器がどこで良好な性能を示し、どこで誤分類するかを特定するのに役立ちます。

2. **各クラスの**ROC**曲線と**AUC**値**:
  - 各クラスの真陽性率と偽陽性率のトレードオフを示す**ROC**（Receiver Operating Characteristic）曲線を表示。
  - **AUC**（Area Under the Curve）値は各クラスの分離可能性を測定し、高い**AUC**値はより良いクラス区別を示します。

3. **特徴量重要度プロット（**FEATURE IMPORTANCE PLOT**）**:
  - XGBoost分類器が使用する最も影響力のある上位10個の特徴量を強調表示。
  - このプロットは、分類プロセスにどの特徴量が最も貢献したかを理解するのに役立ちます。


In [ ]:
# Import necessary libraries for visualization
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve, auc
from xgboost import plot_importance

# Step 1: Confusion Matrix
conf_matrix = confusion_matrix(y_val, xgb_y_pred)
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.title("Confusion Matrix for XGBoost Classifier")
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.show()

# Step 2: ROC Curve and AUC for each class (One-vs-Rest)
y_val_binary = label_binarize(y_val, classes=sorted(set(y_val)))
xgb_y_pred_proba = xgb_clf.predict_proba(X_val)

plt.figure(figsize=(10, 7))
for i in range(y_val_binary.shape[1]):
    fpr, tpr, _ = roc_curve(y_val_binary[:, i], xgb_y_pred_proba[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f"Class {i} (AUC = {roc_auc:.2f})")

plt.plot([0, 1], [0, 1], "k--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("XGBoost Classifier ROC Curve (One-vs-Rest)")
plt.legend(loc="best")
plt.show()

# Step 3: Feature Importance Plot
plt.figure(figsize=(12, 8))
plot_importance(xgb_clf, max_num_features=10, importance_type="weight")
plt.title("Top 10 Feature Importances for XGBoost Classifier")
plt.show()
